In [1]:
# import libraries needed
import pandas as pd
import numpy as np
import time

from spe_vectorizers import spe_featurizer, spe_featurizer2, atom_featurizer, kmer_featurizer
from kaggle_chem import ecfp_featurizer, oned_featurizer
from rdkit import Chem

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score

### Import Data

Import cleaned and standardized data for the Tox21 NR-AhR assay.  Use train and score datasets as train and test data, respectively.

In [2]:
train = pd.read_csv('../processed_data/nr_ahr_std_train.csv')
#train.head()

train_data = train.std_compounds
train_labels = train.label

print('Train data shape:', train_data.shape)
print('Train labels shape:', train_labels.shape)

active_train = train_data[train_labels == 1].reset_index(drop=True)
inactive_train = train_data[train_labels == 0].reset_index(drop=True)

print('Active compounds:', len(train_labels[train_labels == 1]))
print('Inactive compounds:', len(train_labels[train_labels == 0]))
print('Inactive : Active ~', len(train_labels[train_labels == 0]) // len(train_labels[train_labels == 1]))

Train data shape: (6709,)
Train labels shape: (6709,)
Active compounds: 761
Inactive compounds: 5948
Inactive : Active ~ 7


In [3]:
test = pd.read_csv('../processed_data/nr_ahr_test_std.csv')
#test.head()

test_data = test.std_compounds
test_labels = test.label

print('Test data shape:', test_data.shape)
print('Test labels shape:', test_labels.shape)

print('Active compounds:', len(test_labels[test_labels == 1]))
print('Inactive compounds:', len(test_labels[test_labels == 0]))
print('Inactive : Active ~', len(test_labels[test_labels == 0]) // len(test_labels[test_labels == 1]))

Test data shape: (607,)
Test labels shape: (607,)
Active compounds: 71
Inactive compounds: 536
Inactive : Active ~ 7


#### Classifier and Parameter Grid Lists

Create list of models to run and parameter grids for each for running grid search

In [4]:
bnb = BernoulliNB()
rf = RandomForestClassifier()
logreg = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC()

classifier_list = [bnb, rf, logreg, knn, svm]
classifier_names = ['bnb', 'rf', 'logreg', 'knn', 'svm']

In [5]:
bnb_params = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
rf_params = {'max_depth':[50, 100, None], 'n_estimators': [100, 200, 500]}
logreg_params = {'C': [0.5], 'solver': ['liblinear'], 'multi_class': ['auto']}
knn_params = {'n_neighbors': [1, 2, 3, 4, 5, 10]}
svm_params = {'C': [0.0001, 0.001, 0.01, 0.1], 
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'probability': [True]}

param_list = [bnb_params, rf_params, logreg_params, knn_params, svm_params]

#### Helper functions for running model matrix loops

In [6]:
def grid_searcher(classifier, param_grid, x_train, y_train):
    """Helper function for finding best parameters for each the model
       Uses ROC-AUC to evaluate performance of models"""
    
    grid_search = GridSearchCV(estimator=classifier,
                               param_grid=param_grid,
                               cv=5,
                               n_jobs=1,
                               verbose=2,
                               scoring='recall')
    
    grid_search.fit(x_train, y_train)
    best_params = grid_search.best_params_
    return best_params

In [7]:
def run_model(classifier, x_train, x_test):
    """Helper function for running and evaluating models
       Returns recall for each label and ROC-AUC score"""
    
    trained_classifier = classifier.fit(x_train, train_labels)
    
    # calculate recall for 0 and 1 
    report = classification_report(test_labels, trained_classifier.predict(x_test), output_dict=True)
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    
    # calculate roc-auc score for model
    roc_auc = roc_auc_score(test_labels, trained_classifier.predict_proba(x_test)[:, 1])
    
    return recall_0, recall_1, roc_auc

#### Initialize list for constructing matrix results dataframe

In [8]:
results_list = []

## Run Model Matrix Experiment
### Using Conventional RDKit Methods

Run the list of classifiers chosen using different featurizers and collect the results in a dataframe.

For each featurizer, loop through the list of classifiers:
- Find the best parameters using grid search
- Run the best parameters
- Calculate recall for each label and ROC-AUC score for the model

### ECFP Featurizer

In [9]:
# generate features
x_ecfp, x_ecfp_test = ecfp_featurizer(train, test)

print(x_ecfp.shape)
print(x_ecfp_test.shape)

(6709, 100)
(607, 100)


In [10]:
# find best parameters for each model
ecfp_best_params = []

start = time.time()

for i in range(len(classifier_list)):
    clf = classifier_list[i]
    param_grid = param_list[i]
    
    # run grid search
    ecfp_best_params.append(grid_searcher(clf, param_grid, x_ecfp, train_labels))
    print(f'{clf} grid search done')

tot_time = time.time() - start
print(f'Total time = {tot_time}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................

[CV] END ......................................n_neighbors=1; total time=   0.1s
[CV] END ......................................n_neighbors=1; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ...................

[CV] END ................C=0.1, kernel=rbf, probability=True; total time=   3.7s
[CV] END ................C=0.1, kernel=rbf, probability=True; total time=   3.7s
[CV] END ............C=0.1, kernel=sigmoid, probability=True; total time=   2.1s
[CV] END ............C=0.1, kernel=sigmoid, probability=True; total time=   2.1s
[CV] END ............C=0.1, kernel=sigmoid, probability=True; total time=   2.1s
[CV] END ............C=0.1, kernel=sigmoid, probability=True; total time=   2.2s
[CV] END ............C=0.1, kernel=sigmoid, probability=True; total time=   2.1s
SVC() grid search done
Total time = 630.2934939861298


In [11]:
# best model list
bnb_ecfp = BernoulliNB(**ecfp_best_params[0])
rf_ecfp = RandomForestClassifier(**ecfp_best_params[1])
logreg_ecfp = LogisticRegression(**ecfp_best_params[2])
knn_ecfp = KNeighborsClassifier(**ecfp_best_params[3])
svm_ecfp = SVC(**ecfp_best_params[4])

ecfp_classifier_list = [bnb_ecfp, rf_ecfp, logreg_ecfp, knn_ecfp, svm_ecfp]

In [12]:
for i in range(len(ecfp_classifier_list)):
    clf = ecfp_classifier_list[i]
    clf_name = classifier_names[i]
    best_params = ecfp_best_params[i]
    
    # run best model
    recall_0, recall_1, roc_auc = run_model(clf, x_ecfp, x_ecfp_test)
    
    # make dictionary of metrics
    metrics = {'featurizer': 'ecfp', 'model': clf_name, 'best_params': best_params,
               'recall_0': recall_0, 'recall_1': recall_1, 'roc_auc': roc_auc}
    
    # collect in results list
    results_list.append(metrics)
    
len(results_list)

5

### 1D Featurizer (Molecular Descriptors)

In [13]:
# generate features
x_oned, x_oned_test = oned_featurizer(train, test)

print(x_oned.shape)
print(x_oned_test.shape)

(6709, 9)
(607, 9)


In [14]:
# need to use different parameters for SVM because it has trouble converging.
bnb_params = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
rf_params = {'max_depth':[50, 100, None], 'n_estimators': [100, 200, 500]}
logreg_params = {'C': [0.5], 'solver': ['liblinear'], 'multi_class': ['auto']}
knn_params = {'n_neighbors': [1, 2, 3, 4, 5, 10]}
svm_params = {'C': [0.0001, 0.001, 0.01], 
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'probability': [True]}

param_list = [bnb_params, rf_params, logreg_params, knn_params, svm_params]

In [15]:
# find best parameters for each model
oned_best_params = []

start = time.time()

for i in range(len(classifier_list)):
    clf = classifier_list[i]
    param_grid = param_list[i]
    
    # run grid search
    oned_best_params.append(grid_searcher(clf, param_grid, x_oned, train_labels))
    print(f'{clf} grid search done')

tot_time = time.time() - start
print(f'Total time = {tot_time}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END ........................................alpha=1e-10; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END .......................................alpha=0.0001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................

[CV] END ......................................n_neighbors=1; total time=   0.0s
[CV] END ......................................n_neighbors=1; total time=   0.0s
[CV] END ......................................n_neighbors=1; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ...................

In [16]:
# best model list
bnb_oned = BernoulliNB(**oned_best_params[0])
rf_oned = RandomForestClassifier(**oned_best_params[1])
logreg_oned = LogisticRegression(**oned_best_params[2])
knn_oned = KNeighborsClassifier(**oned_best_params[3])
svm_oned = SVC(**oned_best_params[4])

oned_classifier_list = [bnb_oned, rf_oned, logreg_oned, knn_oned, svm_oned]

In [17]:
for i in range(len(oned_classifier_list)):
    clf = oned_classifier_list[i]
    clf_name = classifier_names[i]
    best_params = oned_best_params[i]
    
    # run best model
    recall_0, recall_1, roc_auc = run_model(clf, x_oned, x_oned_test)
    
    # make dictionary of metrics
    metrics = {'featurizer': 'oned', 'model': clf_name, 'best_params': best_params,
               'recall_0': recall_0, 'recall_1': recall_1, 'roc_auc': roc_auc}
    
    # collect in results list
    results_list.append(metrics)
    
len(results_list)

/Users/lea/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lea/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lea/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lea/opt/anaconda3/

10

## Export results

Make results available for exploration in another notebook

In [18]:
# convert results list to a dataframe
model_matrix_conventional = pd.DataFrame(results_list)

In [21]:
model_matrix_conventional

,featurizer,model,best_params,recall_0,recall_1,roc_auc
0,ecfp,bnb,{'alpha': 1e-10},0.652985,0.887324,0.843835
1,ecfp,rf,"{'max_depth': 100, 'n_estimators': 100}",0.979478,0.253521,0.890740
2,ecfp,logreg,"{'C': 0.5, 'multi_class': 'auto', 'solver': 'l...",0.945896,0.422535,0.858682
3,ecfp,knn,{'n_neighbors': 1},0.912313,0.549296,0.730805
4,ecfp,svm,"{'C': 0.1, 'kernel': 'linear', 'probability': ...",0.977612,0.239437,0.850983
5,oned,bnb,{'alpha': 1e-10},1.000000,0.000000,0.554853
6,oned,rf,"{'max_depth': 50, 'n_estimators': 100}",0.945896,0.239437,0.717180
7,oned,logreg,"{'C': 0.5, 'multi_class': 'auto', 'solver': 'l...",0.768657,0.492958,0.690443
8,oned,knn,{'n_neighbors': 1},0.839552,0.394366,0.616959
9,oned,svm,"{'C': 0.0001, 'kernel': 'linear', 'probability...",1.000000,0.000000,0.791400


In [29]:
# generate csv file for use in further exploration
#model_matrix_conventional.to_csv('../processed_data/model_matrix_conventional.csv')